# LoRA トレーニング Notebook

このノートブックは LoRA ファインチューニングを実行し、生成された重みを保存、さらに推論テストを行うためのものです。以下の手順に従ってセルを順次実行してください。

## 1. ランタイム設定
上部メニュー → ランタイム → ランタイムのタイプを変更 → ハードウェア アクセラレータ：**GPU** を選択

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install diffusers transformers accelerate peft safetensors

In [ ]:
from huggingface_hub import login
login(token='YOUR_HUGGINGFACE_TOKEN')

In [ ]:
from diffusers import StableDiffusionPipeline
from peft import LoraConfig, get_peft_model, TaskType

# ベースモデルと LoRA 設定
base_model = 'runwayml/stable-diffusion-v1-5'
pipe = StableDiffusionPipeline.from_pretrained(base_model, use_auth_token=True)
peft_config = LoraConfig(
    task_type=TaskType.TEXT_TO_IMAGE,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
lora_model = get_peft_model(pipe.unet, peft_config)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/LoRA/checkpoints',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_train_steps=200,
    learning_rate=1e-4,
    save_steps=50,
)

In [ ]:
# データセット読み込み例
from torchvision import datasets, transforms

dataset = datasets.ImageFolder(
    '/content/drive/MyDrive/LoRA/dataset',
    transform=transforms.Compose([
        transforms.Resize((512,512)),
        transforms.ToTensor(),
    ])
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=dataset,
)
trainer.train()

In [ ]:
# LoRA 重みの保存
lora_model.save_pretrained('/content/drive/MyDrive/LoRA/output')

In [ ]:
# 推論テスト
pipe.unet.load_attn_procs('/content/drive/MyDrive/LoRA/output')
image = pipe('your prompt here').images[0]
image.save('/content/drive/MyDrive/LoRA/sample.png')